In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import argparse
from scipy.stats import gaussian_kde
import os
import yaml
import h5py
import pandas as pd
import openpyxl
from openpyxl import load_workbook
import time

import warnings

In [ ]:
import sys
sys.path.append(r'../tools_F-J/')
sys.path.append(r'../tools_F-J/toollib_DisbaCode')
from objective_function import (ObjectiveFunctionDerivativeUsed, Forward)
from toollib_DisbaCode import forward
from toollib_DisbaCode import objective_function
from toollib_DisbaCode import empirical_relation
from toollib_DisbaCode import plot_disp
from toollib_DisbaCode import plot_model
from toollib_DisbaCode import plot_lcurve
from toollib_DisbaCode import plot_inversion    
from toollib_DisbaCode import plot_kernel

#from toollib_standard import maplib
#from toollib_standard import mathlib
#from toollib_standard import filelib
#from toollib_standard import stacklib
from toollib_standard import plotlib

In [ ]:
flag_project = 1 # 0--regular; 1--repartrition

In [ ]:
if flag_project == 0:
    with open('a-project.yml', 'r', encoding='utf-8') as f:
        proj = yaml.load(f.read(), Loader=yaml.FullLoader)
    name_project = proj['name']
    #name_project = 'project/output_FJSJ_16-01/'               # Harmon server
elif flag_project == 1:
    with open('a-project_repar.yml', 'r', encoding='utf-8') as f:
        proj = yaml.load(f.read(), Loader=yaml.FullLoader)
    name_project = proj['name']
    #name_project = 'project_repartrition/repartrition_01-03/'               # Harmon server

In [ ]:
with open('0_config.yml', 'r', encoding='utf-8') as f:
    dir_config = yaml.load(f.read(), Loader=yaml.FullLoader)
dir_project_workspace = dir_config['dir_project_workspace']
dir_CC_workspace = dir_config['dir_CC_workspace']
print('dir_CC_workspace: ', dir_CC_workspace)
print('dir_project_workspace: ', dir_project_workspace)
dir_project = os.path.join(dir_project_workspace, name_project)
print('dir_project: ', dir_project)

In [ ]:
filename = dir_project+'Basic_info.yml'
with open(filename, 'r', encoding='utf-8') as f:
    info_basic = yaml.load(f.read(), Loader=yaml.FullLoader)
filename_bi = dir_project+'Basic_info.npy'
info_basic_bi = np.load(filename_bi, allow_pickle='TRUE').item()      # setting dictionary

In [ ]:
dir_ds = dir_project + info_basic['dir_ds']
dir_partition = dir_project + info_basic['dir_partition']
dir_inv_dispernet = dir_project + info_basic['dir_inv_dispernet'] 
d_len = info_basic['d_len']
f = info_basic_bi['f']
c = np.linspace(info_basic['c_min'], info_basic['c_max'], info_basic['c_num'])

In [ ]:
nums = [2]
key_subworks_all = info_basic['key_subworks']
key_subworks = []
for num in nums:
    for key_subwork in key_subworks_all:
        if str(num) == key_subwork.split('--')[0]:
            key_subworks.append(key_subwork)
            break
key_subworks = info_basic['key_subworks'][57:]
key_subworks

In [ ]:
file_config_fund = 'config_inv_fund.yml'
file_config = 'config_inv.yml'
flag_forward = 0

In [ ]:
dir_image = dir_project +  info_basic['dir_image'] + 'inversion_compare/'
#info_basic['dir_image_inv'] = info_basic['dir_inv_dispernet'] + 'image_inv_' + info_basic['tag'] + '/'
info_basic['dir_invfile'] = info_basic['dir_inv_dispernet'] + 'inversion/'
info_basic['dir_disp_model'] = info_basic['dir_inv_dispernet'] + 'disp_model/'
info_basic['dir_model'] = info_basic['dir_inv_dispernet'] + 'model/'
dir_invfile = dir_project + info_basic['dir_invfile']
dir_disp_model = dir_project + info_basic['dir_disp_model']
dir_model = dir_project + info_basic['dir_model']

#dir_image_fund = info_basic['dir_inv_dispernet'] + 'image_inv_fund_' + info_basic['tag'] + '/'
info_basic['dir_image_inv_fund'] = info_basic['dir_inv_dispernet'] + 'image_inv_fund_' + info_basic['tag'] + '/'
info_basic['dir_invfile_fund'] = info_basic['dir_inv_dispernet'] + 'inversion_fund/'
info_basic['dir_disp_model_fund'] = info_basic['dir_inv_dispernet'] + 'disp_model_fund/'
info_basic['dir_model_fund'] = info_basic['dir_inv_dispernet'] + 'model_fund/'
dir_invfile_fund = dir_project + info_basic['dir_invfile_fund']
dir_disp_model_fund = dir_project + info_basic['dir_disp_model_fund']
dir_model_fund = dir_project + info_basic['dir_model_fund']
if not os.path.exists(dir_image):
    os.mkdir(dir_image)
if not os.path.exists(dir_invfile):
    os.mkdir(dir_invfile)
if not os.path.exists(dir_disp_model):
    os.mkdir(dir_disp_model)
if not os.path.exists(dir_model):
    os.mkdir(dir_model)

if not os.path.exists(dir_invfile_fund):
    os.mkdir(dir_invfile_fund)
if not os.path.exists(dir_disp_model_fund):
    os.mkdir(dir_disp_model_fund)
if not os.path.exists(dir_model_fund):
    os.mkdir(dir_model_fund)

### Forard and plot estimated disp and Vs

In [ ]:
stalistname_all = info_basic['stalistname_all']
stainfo = pd.read_excel(stalistname_all)
nsta_all = len(stainfo.iloc[:,0])
stalist_all = stainfo['Station'].tolist()
lat_all = stainfo['latitude'].tolist() 
lon_all = stainfo['longitude'].tolist()
stainfo_all = stainfo

In [ ]:
# fault surface trace
faults = np.load('clark_faults.npy', allow_pickle='TRUE').item()

In [ ]:
stations = {}
lat = {}
lon = {}
for key in key_subworks:
    filepath = dir_partition + str(key) + '.txt'
    stations_this, lat_stations_this, lon_stations_this = np.loadtxt(filepath, dtype='str' , unpack=True)
    stations[key] = stations_this
    lat[key] = lat_stations_this.astype(float)
    lon[key] = lon_stations_this.astype(float)

In [ ]:
def plot_fj(ax,ds_linear,title0,f,c,index,v_min,v_max,c_map):
    ax.imshow(np.flip(ds_linear,0),aspect='auto',extent=[min(f),max(f),min(c),max(c)],vmin=v_min,vmax = v_max, cmap = c_map)
    #plt.imshow(np.flip(ds_linear,0),extent=[min(f),max(f),min(c),max(c)],aspect='auto',cmap='jet',vmin=0,vmax=1)
    #ax.pcolormesh(f,c,ds_linear,cmap='jet',vmin=0,vmax=1)
    if index == 0:
        ax.set_title(title0)
    else:
        ax.set_title('('+chr(96+index)+')',loc='left')
    ax.set_xlabel('Normalized Frequency/ Hz')
    ax.set_ylabel('Velocity/ m/s')
    #ax.set_xlim(xlim)
    return ax

In [ ]:
dir_this = os.getcwd()
dir_this

In [ ]:
#%%capture
#key_subworks = ['25-06']
c_map = 'jet'
v_max = None
v_min = 0.1

#s.chdir('/home/harmon/data/git_repo/F-J_Workflow/San_Jasinto/'+dir_project+'inv_dispernet')

with open(dir_project + info_basic['dir_inv_dispernet']+file_config, 'r') as fp:
    config = yaml.safe_load(fp)

with open(dir_inv_dispernet + 'initial/initial_tag.yaml', 'r') as fp:
    tag = yaml.safe_load(fp)


all_disp = 0
show_init = 0
plot_fund = 1
xlim_f = [0.5,30]
xlim_T = [-1,1]
ylim_c = [0.25,1.8]
clim = [0,1]
fmin = 1
fmax = 40
cmin = 0.1
cmax = 2.5
vmax = 0.3

flag_num = 0
for key_subwork in key_subworks:
    flag_num += 1
    print(key_subwork)
    fig,ax = plt.subplots(3,3,figsize=(27,20))
    #plt.figure(figsize=(27,20))
    #ax = plt.subplot2grid((1,3),(0,0),colspan=2,rowspan=1)

    data_file = 'data/ds_'+key_subwork+'curve.txt'
    file_model = 'model/model_'+key_subwork+'.txt'
    file_fund = 'model_fund/model_'+key_subwork+'.txt'
    file_inv =  'ds_'+key_subwork+'curve/'
    disp_forward = dir_disp_model +  'disp_model_'+key_subwork+'.txt'
    disp_forward_fund = dir_disp_model_fund +  'disp_model_'+key_subwork+'.txt'
    if not os.path.exists(dir_invfile+file_inv):
        print(key_subwork+'data file not exist')
        plt.close()
        continue
    


    ds = h5py.File(dir_ds+'ds_'+key_subwork+'.h5', 'r')
    ds_remove = ds['ds_remove'][0]
    ds_remove = plotlib.smooth_ds(ds_remove)
    title0 = "Fundamental"
    #ax[0][0].set_xlim(xlim_f)
    #ax[0][0].set_ylim(ylim_c)
    #ax[0][0] = plot_fj(ax[0][0],ds_remove,title0,f,c,xlim_f,0,v_min,v_max,c_map)

    title0 = "Overtones"
    ax[0][1].set_xlim(xlim_f)
    ax[0][1].set_ylim(ylim_c)
    #ax[0] = plot_fj(ax[0],ds_remove,title0,f,c,xlim_f,0,clim)
    ax[0][1] = plot_fj(ax[0][1],ds_remove,title0,f,c,0,v_min,v_max,c_map)

    os.chdir(dir_project_workspace +'/'+name_project+'/inv_dispernet/')
    disp_data = np.loadtxt(data_file)
    data_name = 'ds_'+key_subwork+'curve'
    ax[0][2] = plot_inversion.plot_disp(config, data_name, all_disp,ax[0][2])
    # s wave velocity inversion
    
    #file_init = 'initial/initial_model_'+str(tag[key_subwork])+'.txt'
    file_init = 'initial/initial_model_4.txt'
    ax[1][1],model,fmt = plot_inversion.plot_model(config, data_name, show_init, file_model,file_init,ax[1][1],file_fund,plot_fund)
    #ax[1][1],model,fmt = plot_inversion.plot_model(config, data_name, show_init, file_model,file_init,ax[1][1],file_fund,plot_fund)
    os.chdir(dir_this + '/')

    if flag_forward == 1:
        max_mode = 4
        forward.forward_cal(info_basic,max_mode,dir_inv_dispernet+file_model,filename)
        forward.forward_cal(info_basic,max_mode,dir_inv_dispernet+file_fund,filename)


    disp_model = np.loadtxt(disp_forward)
    disp_model_fund = np.loadtxt(disp_forward_fund)
    modes = set(disp_data[:, 2].astype(int))

    for m in modes:
        d = disp_data[disp_data[:, 2] == m]
        if m==0:
            #ax[0][0].plot(d[:, 0], d[:, 1], 'g^',label='data')
            ax[0][1].plot(d[:, 0], d[:, 1], 'k^',label='data',linewidth=4)
        else:
            #ax[0][0].plot(d[:, 0], d[:, 1], 'g^')
            ax[0][1].plot(d[:, 0], d[:, 1], 'k^',linewidth=4 )

    modes = set(disp_model[:, 2].astype(int))
    for m in modes:
        d = disp_model[disp_model[:, 2] == m]
        if m==0:
            ax[0][1].plot(d[:, 0], d[:, 1], color='greenyellow',linestyle = '--',label='estemate',linewidth=2)
        else:
            ax[0][1].plot(d[:, 0], d[:, 1], color='greenyellow',linestyle = '--',linewidth=2)
    modes = set(disp_model_fund[:, 2].astype(int))
    for m in modes:
        d = disp_model_fund[disp_model_fund[:, 2] == m]
        """
        if m==0:
            ax[0][0].plot(d[:, 0], d[:, 1], 'r--',label='estemate',linewidth=5)
        else:
            ax[0][0].plot(d[:, 0], d[:, 1], 'r--',linewidth=5)
        """
    #ax[0][0].legend()
    ax[0][1].legend()

    ax[1][0].axis('off')
    ax[1][2].axis('off')



    file_disp = dir_project + info_basic['dir_inv_dispernet']+'data/ds_'+key_subwork+'curve.txt'
    file_model = dir_model+'model_'+key_subwork+'.txt'
    disp = np.loadtxt(file_disp)
    m = int(max(set(disp[:,2])))
    if m > 2:
        m = 2
    for mode in range(m+1):
        #print(mode)
        #ax[2][mode] = plot_kernel.kernel_plot(fmin,fmax,cmin,cmax,file_disp,file_model,mode,vmax,ax[2][mode])
        
        try:
            ax[2][mode] = plot_kernel.kernel_plot(fmin,fmax,cmin,cmax,file_disp,file_model,mode,vmax,ax[2][mode])
        except:
            print('error')



    stalist = stations[key_subwork]
    lat_this = lat[key_subwork]
    lon_this = lon[key_subwork]
    nsta = len(stalist)
    ax[0][0]=plotlib.plot_area(ax[0][0],lon_all,lat_all,lon_this,lat_this)
    lw= 5
    for i in range(len(faults)):
        ax[0][0].plot(faults['clark'+str(i+1)]['lon'], faults['clark'+str(i+1)]['lat'], 'k',linewidth=lw)

    """
    key = 'R'+key_subwork[0:2]+key_subwork[3:5]
    if key in stalist_all:
        lat_this = lat_all[stalist_all.index(key)]
        lon_this = lon_all[stalist_all.index(key)]
    else:
        key1 = 'R' + str(int(key_subwork[0:2])-1).zfill(2)+key_subwork[3:5]
        key2 = 'R' + str(int(key_subwork[0:2])+1).zfill(2)+key_subwork[3:5]
        indx1 = stalist_all.index(key1)
        indx2 = stalist_all.index(key2)
        lat_this = (lat_all[indx1]+lat_all[indx2])/2
        lon_this = (lon_all[indx1]+lon_all[indx2])/2
    ax[0][0].scatter(lon_this,lat_this,marker='o',color='g',s=200)
    """

    plt.tight_layout()
    #plt.savefig(dir_image+'Sens-kernel_'+key_subwork+'.png',dpi=60)
    plt.savefig(dir_image+'invplot_all_'+key_subwork+'.png',dpi=100)
    print('finish '+str(flag_num)+'/'+str(len(key_subworks)))
    plt.close()

In [ ]:
with open(dir_project+'Basic_info.yml', 'w', encoding='utf-8') as f:
   yaml.dump(data=info_basic, stream=f, allow_unicode=True)